# ETL Project
## Terrence Cummings
Scraping race mix by Minneapolis neighborhood

In [35]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json
import requests
import pymongo
from splinter import Browser
from selenium import webdriver
import time
import sys
import os
import datetime
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless")

In [36]:
#Read in CSV's from Extract
cmnty_links_df=pd.read_csv('cmnty_links.csv')
nbhd_links_df=pd.read_csv('nbhd_links.csv')
cmnty_race_df=pd.read_csv('cmnty_race.csv')
nbhd_race_df=pd.read_csv('nbhd_race.csv')
cmnty_keys_df=pd.read_csv('cmnty_keys.csv')
nbhd_keys_df=pd.read_csv('nbhd_keys.csv')

In [37]:
nbhd_race_df['neighborhood'] = nbhd_race_df['neighborhood'].str.upper()
nbhd_race_df = nbhd_race_df.replace('suppressed', 'empty')
nbhd_race_df = nbhd_race_df.replace(np.nan, 'empty', regex=True)
nbhd_race_df

,neighborhood,white_cnt,white_pct,black_cnt,black_pct,native_cnt,native_pct,asian_cnt,asian_pct,other_cnt,other_pct,two_or_more_cnt,two_or_more_pct,hispanic_cnt,hispanic_pct,of_color_cnt,of_color_pct,URL
0,ARMATAGE,"4,316",84.5%,empty,empty,empty,empty,empty,empty,empty,empty,185,3.6%,empty,empty,792,15.5%,http://www.mncompass.org/profiles/neighborhood...
1,AUDUBON-PARK,"4,034",78.8%,386,7.5%,empty,empty,empty,empty,empty,empty,303,5.9%,449,8.8%,"1,083",21.2%,http://www.mncompass.org/profiles/neighborhood...
2,BANCROFT,"2,601",73.4%,413,11.6%,empty,empty,74,2.1%,empty,empty,278,7.8%,521,14.7%,943,26.6%,http://www.mncompass.org/profiles/neighborhood...
3,CEDAR-ISLES-DEAN,"2,927",91.6%,empty,empty,empty,empty,empty,empty,empty,empty,empty,empty,empty,empty,empty,empty,http://www.mncompass.org/profiles/neighborhood...
4,CEDAR-RIVERSIDE,"3,219",32.0%,"5,146",51.1%,empty,empty,"1,156",11.5%,empty,empty,396,3.9%,270,2.7%,"6,846",68.0%,http://www.mncompass.org/profiles/neighborhood...
5,CENTRAL,"3,567",43.3%,"2,116",25.7%,empty,empty,194,2.4%,"1,738",21.1%,513,6.2%,"3,204",38.9%,"4,670",56.7%,http://www.mncompass.org/profiles/neighborhood...
6,COMO,"4,314",72.2%,552,9.2%,empty,empty,893,14.9%,empty,empty,106,1.8%,427,7.1%,"1,663",27.8%,http://www.mncompass.org/profiles/neighborhood...
7,COOPER,"3,009",81.4%,empty,empty,empty,empty,empty,empty,empty,empty,113,3.1%,empty,empty,686,18.6%,http://www.mncompass.org/profiles/neighborhood...
8,CORCORAN,"2,849",63.6%,760,17.0%,empty,empty,115,2.6%,352,7.9%,253,5.6%,920,20.6%,"1,630",36.4%,http://www.mncompass.org/profiles/neighborhood...
9,DIAMOND-LAKE,"4,429",76.8%,651,11.3%,empty,empty,empty,empty,empty,empty,empty,empty,485,8.4%,"1,335",23.2%,http://www.mncompass.org/profiles/neighborhood...


In [38]:
cmnty_race_df['community'] = cmnty_race_df['community'].str.upper()
cmnty_race_df = cmnty_race_df.replace('suppressed', '0')
cmnty_race_df = cmnty_race_df.replace(np.nan, '0', regex=True)
cmnty_race_df

,community,white_cnt,white_pct,black_cnt,black_pct,native_cnt,native_pct,asian_cnt,asian_pct,other_cnt,other_pct,two_or_more_cnt,two_or_more_pct,hispanic_cnt,hispanic_pct,of_color_cnt,of_color_pct,URL
0,CALHOUN-ISLES,"27,911",86.1%,"1,591",4.9%,0,0,"1,080",3.3%,416,1.3%,"1,309",4.0%,"1,166",3.6%,"4,512",13.9%,http://www.mncompass.org/profiles/communities/...
1,CAMDEN,"13,565",44.1%,"10,677",34.7%,343,1.1%,"3,500",11.4%,691,2.2%,"1,953",6.4%,"2,360",7.7%,"17,164",55.9%,http://www.mncompass.org/profiles/communities/...
2,CENTRAL,"23,384",66.2%,"7,133",20.2%,405,1.1%,"2,410",6.8%,410,1.2%,"1,564",4.4%,"1,645",4.7%,"11,928",33.8%,http://www.mncompass.org/profiles/communities/...
3,LONGFELLOW,"20,399",70.3%,"5,521",19.0%,290,1.0%,815,2.8%,901,3.1%,"1,072",3.7%,"2,083",7.2%,"8,613",29.7%,http://www.mncompass.org/profiles/communities/...
4,NEAR-NORTH,"8,264",23.0%,"17,163",47.9%,629,1.8%,"5,493",15.3%,"1,864",5.2%,"2,448",6.8%,"3,784",10.5%,"27,604",77.0%,http://www.mncompass.org/profiles/communities/...
5,NOKOMIS,"30,885",79.6%,"3,507",9.0%,629,1.6%,776,2.0%,"1,214",3.1%,"1,779",4.6%,"2,160",5.6%,"7,905",20.4%,http://www.mncompass.org/profiles/communities/...
6,NORTHEAST,"28,051",74.9%,"3,921",10.5%,674,1.8%,"1,035",2.8%,"1,999",5.3%,"1,793",4.8%,"3,989",10.6%,"9,422",25.1%,http://www.mncompass.org/profiles/communities/...
7,PHILLIPS,"6,310",28.6%,"9,606",43.5%,"1,255",5.7%,757,3.4%,"3,400",15.4%,750,3.4%,"5,709",25.9%,"15,768",71.4%,http://www.mncompass.org/profiles/communities/...
8,POWDERHORN,"33,738",59.0%,"11,305",19.8%,787,1.4%,"1,387",2.4%,"7,056",12.3%,"2,876",5.0%,"13,036",22.8%,"23,438",41.0%,http://www.mncompass.org/profiles/communities/...
9,SOUTHWEST,"42,747",86.0%,"2,289",4.6%,0,0,"1,650",3.3%,959,1.9%,"1,842",3.7%,"2,021",4.1%,"6,986",14.0%,http://www.mncompass.org/profiles/communities/...


In [39]:
cmnty_race_df['white_cnt'] = cmnty_race_df['white_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['black_cnt'] = cmnty_race_df['black_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['native_cnt'] = cmnty_race_df['native_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['asian_cnt'] = cmnty_race_df['asian_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['other_cnt'] = cmnty_race_df['other_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['two_or_more_cnt'] = cmnty_race_df['two_or_more_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['hispanic_cnt'] = cmnty_race_df['hispanic_cnt'].str.replace(',', '').astype(float)
cmnty_race_df['of_color_cnt'] = cmnty_race_df['of_color_cnt'].str.replace(',', '').astype(float)


In [40]:
cmnty_race_df['white_pct'] = cmnty_race_df['white_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['black_pct'] = cmnty_race_df['black_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['native_pct'] = cmnty_race_df['native_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['asian_pct'] = cmnty_race_df['asian_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['other_pct'] = cmnty_race_df['other_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['two_or_more_pct'] = cmnty_race_df['two_or_more_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['hispanic_pct'] = cmnty_race_df['hispanic_pct'].str.replace('%', '').astype(float)/100
cmnty_race_df['of_color_pct'] = cmnty_race_df['of_color_pct'].str.replace('%', '').astype(float)/100

In [41]:
cmnty_race_df

,community,white_cnt,white_pct,black_cnt,black_pct,native_cnt,native_pct,asian_cnt,asian_pct,other_cnt,other_pct,two_or_more_cnt,two_or_more_pct,hispanic_cnt,hispanic_pct,of_color_cnt,of_color_pct,URL
0,CALHOUN-ISLES,27911.0,0.861,1591.0,0.049,0.0,0.000,1080.0,0.033,416.0,0.013,1309.0,0.040,1166.0,0.036,4512.0,0.139,http://www.mncompass.org/profiles/communities/...
1,CAMDEN,13565.0,0.441,10677.0,0.347,343.0,0.011,3500.0,0.114,691.0,0.022,1953.0,0.064,2360.0,0.077,17164.0,0.559,http://www.mncompass.org/profiles/communities/...
2,CENTRAL,23384.0,0.662,7133.0,0.202,405.0,0.011,2410.0,0.068,410.0,0.012,1564.0,0.044,1645.0,0.047,11928.0,0.338,http://www.mncompass.org/profiles/communities/...
3,LONGFELLOW,20399.0,0.703,5521.0,0.190,290.0,0.010,815.0,0.028,901.0,0.031,1072.0,0.037,2083.0,0.072,8613.0,0.297,http://www.mncompass.org/profiles/communities/...
4,NEAR-NORTH,8264.0,0.230,17163.0,0.479,629.0,0.018,5493.0,0.153,1864.0,0.052,2448.0,0.068,3784.0,0.105,27604.0,0.770,http://www.mncompass.org/profiles/communities/...
5,NOKOMIS,30885.0,0.796,3507.0,0.090,629.0,0.016,776.0,0.020,1214.0,0.031,1779.0,0.046,2160.0,0.056,7905.0,0.204,http://www.mncompass.org/profiles/communities/...
6,NORTHEAST,28051.0,0.749,3921.0,0.105,674.0,0.018,1035.0,0.028,1999.0,0.053,1793.0,0.048,3989.0,0.106,9422.0,0.251,http://www.mncompass.org/profiles/communities/...
7,PHILLIPS,6310.0,0.286,9606.0,0.435,1255.0,0.057,757.0,0.034,3400.0,0.154,750.0,0.034,5709.0,0.259,15768.0,0.714,http://www.mncompass.org/profiles/communities/...
8,POWDERHORN,33738.0,0.590,11305.0,0.198,787.0,0.014,1387.0,0.024,7056.0,0.123,2876.0,0.050,13036.0,0.228,23438.0,0.410,http://www.mncompass.org/profiles/communities/...
9,SOUTHWEST,42747.0,0.860,2289.0,0.046,0.0,0.000,1650.0,0.033,959.0,0.019,1842.0,0.037,2021.0,0.041,6986.0,0.140,http://www.mncompass.org/profiles/communities/...


In [42]:
cmnty_dict = {'CALHOUN-ISLES': 'CALHOUN-ISLE', 'CAMDEN':'CAMDEN', 'CENTRAL': 'CENTRAL', 'LONGFELLOW': 'LONGFELLOW', 'NEAR-NORTH': 'NEAR NORTH', 'NOKOMIS':'NOKOMIS', 'NORTHEAST':'NORTHEAST', 'PHILLIPS': 'PHILLIPS', 'POWDERHORN': 'POWDERHORN', 'SOUTHWEST':'SOUTHWEST', 'UNIVERSITY':'UNIVERSITY'}

In [43]:
cmnty_race_df = cmnty_race_df.replace({"community": cmnty_dict})

In [44]:
cmnty_race_df

,community,white_cnt,white_pct,black_cnt,black_pct,native_cnt,native_pct,asian_cnt,asian_pct,other_cnt,other_pct,two_or_more_cnt,two_or_more_pct,hispanic_cnt,hispanic_pct,of_color_cnt,of_color_pct,URL
0,CALHOUN-ISLE,27911.0,0.861,1591.0,0.049,0.0,0.000,1080.0,0.033,416.0,0.013,1309.0,0.040,1166.0,0.036,4512.0,0.139,http://www.mncompass.org/profiles/communities/...
1,CAMDEN,13565.0,0.441,10677.0,0.347,343.0,0.011,3500.0,0.114,691.0,0.022,1953.0,0.064,2360.0,0.077,17164.0,0.559,http://www.mncompass.org/profiles/communities/...
2,CENTRAL,23384.0,0.662,7133.0,0.202,405.0,0.011,2410.0,0.068,410.0,0.012,1564.0,0.044,1645.0,0.047,11928.0,0.338,http://www.mncompass.org/profiles/communities/...
3,LONGFELLOW,20399.0,0.703,5521.0,0.190,290.0,0.010,815.0,0.028,901.0,0.031,1072.0,0.037,2083.0,0.072,8613.0,0.297,http://www.mncompass.org/profiles/communities/...
4,NEAR NORTH,8264.0,0.230,17163.0,0.479,629.0,0.018,5493.0,0.153,1864.0,0.052,2448.0,0.068,3784.0,0.105,27604.0,0.770,http://www.mncompass.org/profiles/communities/...
5,NOKOMIS,30885.0,0.796,3507.0,0.090,629.0,0.016,776.0,0.020,1214.0,0.031,1779.0,0.046,2160.0,0.056,7905.0,0.204,http://www.mncompass.org/profiles/communities/...
6,NORTHEAST,28051.0,0.749,3921.0,0.105,674.0,0.018,1035.0,0.028,1999.0,0.053,1793.0,0.048,3989.0,0.106,9422.0,0.251,http://www.mncompass.org/profiles/communities/...
7,PHILLIPS,6310.0,0.286,9606.0,0.435,1255.0,0.057,757.0,0.034,3400.0,0.154,750.0,0.034,5709.0,0.259,15768.0,0.714,http://www.mncompass.org/profiles/communities/...
8,POWDERHORN,33738.0,0.590,11305.0,0.198,787.0,0.014,1387.0,0.024,7056.0,0.123,2876.0,0.050,13036.0,0.228,23438.0,0.410,http://www.mncompass.org/profiles/communities/...
9,SOUTHWEST,42747.0,0.860,2289.0,0.046,0.0,0.000,1650.0,0.033,959.0,0.019,1842.0,0.037,2021.0,0.041,6986.0,0.140,http://www.mncompass.org/profiles/communities/...


In [45]:
cmnty_keys_df

,CMNTY_KEY,COMMUNITY
0,C00,CALHOUN-ISLE
1,C01,CAMDEN
2,C02,CENTRAL
3,C03,LONGFELLOW
4,C04,NEAR NORTH
5,C05,NOKOMIS
6,C06,NORTHEAST
7,C07,PHILLIPS
8,C08,POWDERHORN
9,C09,SOUTHWEST


In [46]:
final_cmnty_race_df = cmnty_race_df.merge(cmnty_keys_df, how='left', left_on='community', right_on='COMMUNITY')
del final_cmnty_race_df['COMMUNITY']
final_cmnty_race_df.set_index('CMNTY_KEY', inplace=True)
final_cmnty_race_df

,community,white_cnt,white_pct,black_cnt,black_pct,native_cnt,native_pct,asian_cnt,asian_pct,other_cnt,other_pct,two_or_more_cnt,two_or_more_pct,hispanic_cnt,hispanic_pct,of_color_cnt,of_color_pct,URL
CMNTY_KEY,,,,,,,,,,,,,,,,,,
C00,CALHOUN-ISLE,27911.0,0.861,1591.0,0.049,0.0,0.000,1080.0,0.033,416.0,0.013,1309.0,0.040,1166.0,0.036,4512.0,0.139,http://www.mncompass.org/profiles/communities/...
C01,CAMDEN,13565.0,0.441,10677.0,0.347,343.0,0.011,3500.0,0.114,691.0,0.022,1953.0,0.064,2360.0,0.077,17164.0,0.559,http://www.mncompass.org/profiles/communities/...
C02,CENTRAL,23384.0,0.662,7133.0,0.202,405.0,0.011,2410.0,0.068,410.0,0.012,1564.0,0.044,1645.0,0.047,11928.0,0.338,http://www.mncompass.org/profiles/communities/...
C03,LONGFELLOW,20399.0,0.703,5521.0,0.190,290.0,0.010,815.0,0.028,901.0,0.031,1072.0,0.037,2083.0,0.072,8613.0,0.297,http://www.mncompass.org/profiles/communities/...
C04,NEAR NORTH,8264.0,0.230,17163.0,0.479,629.0,0.018,5493.0,0.153,1864.0,0.052,2448.0,0.068,3784.0,0.105,27604.0,0.770,http://www.mncompass.org/profiles/communities/...
C05,NOKOMIS,30885.0,0.796,3507.0,0.090,629.0,0.016,776.0,0.020,1214.0,0.031,1779.0,0.046,2160.0,0.056,7905.0,0.204,http://www.mncompass.org/profiles/communities/...
C06,NORTHEAST,28051.0,0.749,3921.0,0.105,674.0,0.018,1035.0,0.028,1999.0,0.053,1793.0,0.048,3989.0,0.106,9422.0,0.251,http://www.mncompass.org/profiles/communities/...
C07,PHILLIPS,6310.0,0.286,9606.0,0.435,1255.0,0.057,757.0,0.034,3400.0,0.154,750.0,0.034,5709.0,0.259,15768.0,0.714,http://www.mncompass.org/profiles/communities/...
C08,POWDERHORN,33738.0,0.590,11305.0,0.198,787.0,0.014,1387.0,0.024,7056.0,0.123,2876.0,0.050,13036.0,0.228,23438.0,0.410,http://www.mncompass.org/profiles/communities/...
